# **0-** Background

- **The Notebook does product recommendation analysis. Using Co-occurence matrices, this code creates a co-occurence metric and a transaction metric**
     - Co-occurrence Count : Refers to how many times Product Focus and the Recommended Product have been bought together by the same customer across different transactions (added across all customers).
     - Transaction Count : Refers to how many times Product Focus and the Recommended Product have been bought together in the same transaction (added across all customers).

## **0.1-** Extra

# **1-** Enviroment Setup

In [ ]:
# Install when needed
!pip install openpyxl
!pip install mlxtend

In [6]:
import pandas as pd
import numpy as np
from itertools import combinations, product
from collections import defaultdict

# **2-** Data Input (GCS) & Cleaning

In [7]:
# Read the Excel file into a dataframe from GCS/local
data = pd.read_excel("/home/user/MyCodeFiles/PR/YSL - Products May YTD 2024 - cleaned.xlsx", engine='openpyxl') 

In [111]:
# ONLY IF CLEANED
# Import the cleaned data back if kernel is reset and you dont want to restart the cleaning
data = pd.read_excel("YSL - Products Last 24 Months - cleaned.xlsx", engine='openpyxl') 

In [8]:
data[data.columns]

,custid,transid,date,productcode,productname,qty,amount,Year
0,12044593,001796224,2024-05-13 00:00:00,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
1,12044593,001796224,2024-05-13,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
2,99919582511,001796057,2024-05-13,LA696200,LA696200:LIBRE EDP 90ML,1,183.602295,2024
3,9998248301,001796238,2024-05-13,TAE05955,TAE05955:LB EDP INT 90ML+ LB EDP10ML + LC,1,171.545220,2024
4,9999140813,001795986,2024-05-13,TAE05957,TAE05957:Y 100ML+ NEW Y 50ML+ Y BALM,1,150.481881,2024
...,...,...,...,...,...,...,...,...
47304,9998915223,75702301202401011790,2024-01-01,034816402120,LE399800:YSL PREMIUM MEN POUCH DDP,1,0.000000,2024
47305,9998915223,75702301202401011790,2024-01-01,034816545875,LE027700:YSL NEW MASC EDP V1.2ML ECH,1,0.000000,2024
47306,9998915223,75702301202401011790,2024-01-01,034816545875,LE027700:YSL NEW MASC EDP V1.2ML ECH,1,0.000000,2024
47307,9999227431,001660949,2024-01-01,LA697000,LA697000:SAINT LAURENT NEW FEM S1.2ML SAMP,1,0.000000,2024


In [9]:
# Show all unique values of a column:
sorted(data['amount'].unique())

[0.0,
 0.002178352620830497,
 0.0026661334518494704,
 0.0027229399999999996,
 0.0027229407760381214,
 0.9257998638529612,
 3.0667050000000002,
 20.00025,
 21.78352,
 22.464254999999998,
 23.56432276,
 23.73363,
 23.825725,
 24.087127239999997,
 24.185153079999996,
 24.533640000000002,
 24.72157226,
 25.187195,
 25.333650000000002,
 25.867929999999998,
 25.867937372362153,
 25.990462299999997,
 26.412525527569777,
 26.548665,
 27.637840999999998,
 27.773995915588838,
 27.910134999999997,
 28.00035,
 28.400355,
 28.590878148400275,
 29.200365,
 29.407752,
 29.952348536419333,
 30.496927999999997,
 31.193761386638805,
 31.313818924438394,
 31.449956999999998,
 31.586104,
 31.586113002042207,
 32.267070000000004,
 32.402986,
 32.67528931245746,
 32.933745,
 33.083721,
 33.227082,
 33.32666814811838,
 33.356015,
 33.49217154526889,
 33.56024506466984,
 33.764455999999996,
 34.036759700476516,
 34.303607896528256,
 34.30633083730429,
 34.309044,
 34.309053778080326,
 34.34709600000001,
 34.7

## **2.1-** Removing rows with a specific value in a specific column

In [10]:
# Identify rows where the specified column matches the given value
matching_rows = data["amount"] <= 0
count_matching = matching_rows.sum()

print(f"Rows that match: {count_matching}")

Rows that match: 22827


In [11]:
# Show the dataframe if we removed the matching rows
data[~matching_rows]

,custid,transid,date,productcode,productname,qty,amount,Year
0,12044593,001796224,2024-05-13 00:00:00,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
1,12044593,001796224,2024-05-13,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
2,99919582511,001796057,2024-05-13,LA696200,LA696200:LIBRE EDP 90ML,1,183.602295,2024
3,9998248301,001796238,2024-05-13,TAE05955,TAE05955:LB EDP INT 90ML+ LB EDP10ML + LC,1,171.545220,2024
4,9999140813,001795986,2024-05-13,TAE05957,TAE05957:Y 100ML+ NEW Y 50ML+ Y BALM,1,150.481881,2024
...,...,...,...,...,...,...,...,...
47212,10274880,001660732,2024-01-01,L9301800,L9301800:DESSIN DU REGARD WP 02 RENO,1,35.534367,2024
47213,9996038179,001660507,2024-01-01,L0128700,L0128700:YSL MASC VOL.EFFET FX CILS 03 NG,2,34.309044,2024
47214,10274880,001660732,2024-01-01,LB436100,LB436100:TC VELVET CREAM 216 FG,1,33.356015,2024
47215,99918750055,001661062,2024-01-01,LB436100,LB436100:TC VELVET CREAM 216 FG,1,33.356015,2024


In [12]:
# Remove the rows and return the modified DataFrame
data = data[~matching_rows]

In [13]:
data.reset_index(drop=True, inplace=True)

In [14]:
data[data.columns]

,custid,transid,date,productcode,productname,qty,amount,Year
0,12044593,001796224,2024-05-13 00:00:00,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
1,12044593,001796224,2024-05-13,LB703100,LB703100:LIBRE INTENSE 90ML,1,190.605800,2024
2,99919582511,001796057,2024-05-13,LA696200,LA696200:LIBRE EDP 90ML,1,183.602295,2024
3,9998248301,001796238,2024-05-13,TAE05955,TAE05955:LB EDP INT 90ML+ LB EDP10ML + LC,1,171.545220,2024
4,9999140813,001795986,2024-05-13,TAE05957,TAE05957:Y 100ML+ NEW Y 50ML+ Y BALM,1,150.481881,2024
...,...,...,...,...,...,...,...,...
24477,10274880,001660732,2024-01-01,L9301800,L9301800:DESSIN DU REGARD WP 02 RENO,1,35.534367,2024
24478,9996038179,001660507,2024-01-01,L0128700,L0128700:YSL MASC VOL.EFFET FX CILS 03 NG,2,34.309044,2024
24479,10274880,001660732,2024-01-01,LB436100,LB436100:TC VELVET CREAM 216 FG,1,33.356015,2024
24480,99918750055,001661062,2024-01-01,LB436100,LB436100:TC VELVET CREAM 216 FG,1,33.356015,2024


In [ ]:
# If needed
data.drop('Item Quantity', axis=1, inplace=True)

In [77]:
# Save the DataFrame to an Excel file if needed
data.to_excel('YSL - Products May YTD 2024 - cleaned.xlsx', engine='openpyxl', index=False)

# **3-** Analysis

In [116]:
print(data['productcode'].value_counts())

3474636613908    2777
3474636858033    1466
3474636857883    1301
3474636858002    1281
3474636397983    1249
                 ... 
3474636382668       1
3474637052201       1
3474636349913       1
3474636397372       1
3474636494934       1
Name: productcode, Length: 228, dtype: int64


In [117]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS

# AND

# ANOTHER METRIC IS TRANSACTION COUNT = WHERE IT COUNTS HOW MANY TIMES THE PAIR APPEARED IN THE SAME TRANSACTION FOR THE SAME CUSTOMER


# Group by Invoice Number and Speedbus ID to find items bought in the same transaction
transaction_items = data.groupby(['transid', 'custid'])['productcode'].apply(list)

# Calculate co-occurrence within the same transaction
transaction_co_occurrence = defaultdict(int)
for items in transaction_items:
    for item_pair in combinations(set(items), 2):
        transaction_co_occurrence[item_pair] += 1
        transaction_co_occurrence[item_pair[::-1]] += 1

# Convert transaction co-occurrence counts to a DataFrame
transaction_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in transaction_co_occurrence.items()), columns=['Item A', 'Item B', 'Transaction Count'])

# integrate the original co-occurrence count
# Calculate co-occurrence across all customer purchases, regardless of transaction
customer_purchases = data.groupby('custid')['productcode'].apply(list)
overall_co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        overall_co_occurrence[item_pair] += 1
        overall_co_occurrence[item_pair[::-1]] += 1

# Convert overall co-occurrence counts to a DataFrame
overall_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in overall_co_occurrence.items()), columns=['Item A', 'Item B', 'Co-occurrence Count'])

# Merge the two datasets on Item A and Item B
merged_df = pd.merge(transaction_co_occurrence_df, overall_co_occurrence_df, on=['Item A', 'Item B'])

# Normalize scores (optional step here based on the new counts)
# You can normalize either or both count columns as needed

# Sort and select top 5 recommendations based on Co-occurrence Count
recommendations = merged_df.sort_values(by=['Item A', 'Co-occurrence Count', 'Transaction Count'], ascending=[True, False, False]).groupby('Item A').head(5)

In [118]:
# Export the recommendations to a CSV file
# recommendations.to_csv('product_bundling_recommendations_with_co_occurrences.csv', index=False)
recommendations.to_excel('GCC-KER-Raw-Data-full-cleaned-2023-bundling.xlsx', engine='openpyxl', index=False)

# **A-** Other

In [123]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS


# Create a mapping of Speedbus ID to items purchased
customer_purchases = data.groupby('Speedbus ID')['Item ID'].apply(list)

# Calculate co-occurrence of each item pair within the same transaction
co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        co_occurrence[item_pair] += 1
        co_occurrence[item_pair[::-1]] += 1

# Convert the co-occurrence counts to a DataFrame
co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in co_occurrence.items()), columns=['Item A', 'Item B', 'Count'])

# You may want to normalize the scores here based on your specific criteria
# For now, let's assume we're directly using counts as a proxy for scoring

# To normalize scores between 0 and 1, find the max count
max_count = co_occurrence_df['Count'].max()
co_occurrence_df['Score'] = co_occurrence_df['Count'] / max_count

# Now, for each item, select the top 5 items to bundle with based on the score
recommendations = co_occurrence_df.sort_values(by=['Item A', 'Score'], ascending=[True, False]).groupby('Item A').head(5)

# Export the recommendations to a CSV file
recommendations.to_csv('product_bundling_recommendations.csv', index=False)


In [129]:
# THIS CODE CALCS CO-OCCURENCE BASED ON CUSTOMER HISTORICAL PURCHASES = HOW MANY 
# TIMES TWO PRODUCTS APPEARED IN THE SAME HISTORY OF A CUSTOMER COUNTED ACCROSS ALL CUSTOMERS

# AND

# ANOTHER METRIC IS TRANSACTION COUNT = WHERE IT COUNTS HOW MANY TIMES THE PAIR APPEARED IN THE SAME TRANSACTION FOR THE SAME CUSTOMER


# Group by Invoice Number and Speedbus ID to find items bought in the same transaction
transaction_items = data.groupby(['Invoice Number', 'Speedbus ID'])['Item ID'].apply(list)

# Calculate co-occurrence within the same transaction
transaction_co_occurrence = defaultdict(int)
for items in transaction_items:
    for item_pair in combinations(set(items), 2):
        transaction_co_occurrence[item_pair] += 1
        transaction_co_occurrence[item_pair[::-1]] += 1

# Convert transaction co-occurrence counts to a DataFrame
transaction_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in transaction_co_occurrence.items()), columns=['Item A', 'Item B', 'Transaction Count'])

# Now, integrate the original co-occurrence count
# Calculate co-occurrence across all customer purchases, regardless of transaction
customer_purchases = data.groupby('Speedbus ID')['Item ID'].apply(list)
overall_co_occurrence = defaultdict(int)
for items in customer_purchases:
    for item_pair in combinations(set(items), 2):
        overall_co_occurrence[item_pair] += 1
        overall_co_occurrence[item_pair[::-1]] += 1

# Convert overall co-occurrence counts to a DataFrame
overall_co_occurrence_df = pd.DataFrame(((pair[0], pair[1], count) for pair, count in overall_co_occurrence.items()), columns=['Item A', 'Item B', 'Co-occurrence Count'])

# Merge the two datasets on Item A and Item B
merged_df = pd.merge(transaction_co_occurrence_df, overall_co_occurrence_df, on=['Item A', 'Item B'])

# Normalize scores (optional step here based on the new counts)
# You can normalize either or both count columns as needed

# Sort and select top 5 recommendations based on Co-occurrence Count
recommendations = merged_df.sort_values(by=['Item A', 'Co-occurrence Count', 'Transaction Count'], ascending=[True, False, False]).groupby('Item A').head(5)

# Export the recommendations to a CSV file
recommendations.to_csv('product_bundling_recommendations_with_co_occurrences.csv', index=False)

## **B-** Old

In [61]:
# Column names can be changed as needed
#user_id_col = 'Speedbus ID'  # Change this to the name of your user ID column
#item_id_col = 'Item ID'  # Change this to the name of your item ID column

In [ ]:
# Library Importing
import pandas as pd
from collections import defaultdict

# Load the dataset from GCS into a Data frame
df = pd.read_csv("My_Product_Performance_v2022.csv")

# Co-Occurance matrix creation
def create_co_occurrence_matrix(df):
    user_items = defaultdict(set)
    co_occurrence = defaultdict(int)

    for index, row in df.iterrows():
        user_items[row["user_id"]].add(row["item_id"])

    for item_set in user_items.values():
        for item1 in item_set:
            for item2 in item_set:
                if item1 != item2:
                    co_occurrence[(item1, item2)] += 1

    return co_occurrence

def find_similar_items(item_id, co_occurrence_matrix):
    similar_items = defaultdict(int)

    for (item1, item2), count in co_occurrence_matrix.items():
        if item1 == item_id or item2 == item_id:
            similar_items[item2 if item1 == item_id else item1] = count

    return dict(sorted(similar_items.items(), key=lambda x: x[1], reverse=True))


def generate_recommendations(user_id, co_occurrence_matrix, df, top_n=5):
    user_items = df[df["user_id"] == user_id]["item_id"].tolist()
    recommendations = []

    similar_items = defaultdict(int)
    for item in user_items:
        item_similarities = find_similar_items(item, co_occurrence_matrix)
        for similar_item, count in item_similarities.items():
            if similar_item not in user_items:
                similar_items[similar_item] += count

    # Sort similar items by count
    sorted_similar_items = dict(sorted(similar_items.items(), key=lambda x: x[1], reverse=True))

    # Generate recommendations
    for similar_item, _ in sorted_similar_items.items():
        recommendations.append(similar_item)
        if len(recommendations) >= top_n:
            break

    return recommendations


def display_co_occurrence_sample(co_occurrence_matrix, sample_size=10):
    print("Sample of item-item co-occurrence matrix:")
    for i, ((item1, item2), count) in enumerate(co_occurrence_matrix.items()):
        if i >= sample_size:
            break
        print(f"({item1}, {item2}): {count}")


def co_occurrence_matrix_to_df(co_occurrence_matrix):
    co_occurrence_list = [{"item1": item1, "item2": item2, "count": count}
                           for (item1, item2), count in co_occurrence_matrix.items()]
    return pd.DataFrame(co_occurrence_list)


def save_co_occurrence_matrix_to_csv(co_occurrence_matrix, output_filename):
    co_occurrence_df = co_occurrence_matrix_to_df(co_occurrence_matrix)
    co_occurrence_df.to_csv(output_filename, index=False)
    print(f"Co-occurrence matrix has been saved to {output_filename}")

# Apply the algorithm
co_occurrence_matrix = create_co_occurrence_matrix(df)

# Display a sample of the co-occurrence matrix
display_co_occurrence_sample(co_occurrence_matrix)

# Export the full co-occurrence matrix to a CSV file
output_filename = "co_occurrence_matrix.csv"
save_co_occurrence_matrix_to_csv(co_occurrence_matrix, output_filename)

# Generate recommendations for a specific user
# user_id = 1669640  # Set the user_id for which you want to generate recommendations

# recommendations = generate_recommendations(user_id, co_occurrence_matrix, df)

# Print the recommendations
# print(f"\nTop {len(recommendations)} recommendations for user {user_id}:")
# for i, rec_item_id in enumerate(recommendations, start=1):
    # print(f"{i}. Item ID: {rec_item_id}")